<a href="https://colab.research.google.com/github/Pri3scilla/FLiT-Data-Science-Mentorship-Projects/blob/main/market_basket_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


 This steps involves getting the neccessary data needed for the analysis. It involves understanding the dataset and, cleaning it from all forms of possible errors ( such as  nulls and wrong data type) removing duplicates, to ensure a quality data for the analysis.

STEP 1:READING THE DATASET: The first thing is to read the dataset as panda dataframe

In [ ]:
#load the dataset into pandas dataframe
df = pd.read_csv('/content/Market Basket Analysis - Groceries_dataset (1).csv')

In [ ]:
#Take a look at the head

df.head()

,Member_number,Date,itemDescription,single_transaction
0,1808,21-07-2015,tropical fruit,1808_21-07-2015
1,2552,05-01-2015,whole milk,2552_05-01-2015
2,2300,19-09-2015,pip fruit,2300_19-09-2015
3,1187,12-12-2015,other vegetables,1187_12-12-2015
4,3037,01-02-2015,whole milk,3037_01-02-2015


In [ ]:
df.tail()

,Member_number,Date,itemDescription
38760,4471,08-10-2014,sliced cheese
38761,2022,23-02-2014,candy
38762,1097,16-04-2014,cake bar
38763,1510,03-12-2014,fruit/vegetable juice
38764,1521,26-12-2014,cat food


In [ ]:
df.sample(5)

,Member_number,Date,itemDescription
36998,1136,03-01-2015,brown bread
27016,4944,20-01-2014,salty snack
24881,3705,17-08-2014,coffee
23573,2891,02-07-2015,domestic eggs
36415,4913,01-08-2015,brown bread


In [ ]:
df.isnull().sum() #isna

Member_number      0
Date               0
itemDescription    0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


In [ ]:
# To remove excess white space if any is present
df= df.apply(lambda x: x.str.strip() if x.dtype== "object" else x)


In [ ]:
# convert 'Date' column to a standard date format
df['Date'] = pd.to_datetime(df['Date'])

# Ensure date types are appropriate
df.dtypes

Member_number               int64
Date               datetime64[ns]
itemDescription            object
dtype: object

In [ ]:
#check for duplicate rows
duplicate_rows = df[df.duplicated()]
print(duplicate_rows)
#percentage of duplicate values
print(f'number of duplicate entries{df.duplicated().sum()}, duplicate entries in percent {round(df.duplicated().sum()/len(df)*100,2)}%/n')

       Member_number       Date   itemDescription
5015            2051 2015-11-09       frankfurter
5022            3055 2015-08-18  other vegetables
5044            1994 2015-11-03        whole milk
5055            1682 2015-06-25         pip fruit
5059            4324 2015-05-01           sausage
...              ...        ...               ...
38614           2027 2014-02-26     domestic eggs
38684           2936 2014-07-03        newspapers
38685           2311 2014-03-13        pot plants
38722           3834 2014-05-18       salty snack
38723           1146 2014-05-23            yogurt

[759 rows x 3 columns]
number of duplicate entries759, duplicate entries in percent 1.96%/n


In [ ]:
#Remove duplicates from the original data frame
df = df.drop_duplicates()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Check again for duplicates
print(df.duplicated().sum())

759


STEP 2 Data preparation for Market Basket Aalysis: Before we perform market basket analysis, we need to convert this data into a format that can easily be ingested into the Apriori algorithm. i.e, we need to turn it into a tabular structure comprising ones and Zeros.To do this, we first group items that have the same member number and date. That is, member number and date are under thesame column "single transaction". The “single_transaction” variable combines the member number, and date, and tells us the item purchased in one receipt.

In [ ]:
#Group items with the same member number and date
df['single_transaction'] = df['Member_number'].astype(str)+'_'+df['Date'].astype(str)

df.head()







,Member_number,Date,itemDescription,single_transaction
0,1808,21-07-2015,tropical fruit,1808_21-07-2015
1,2552,05-01-2015,whole milk,2552_05-01-2015
2,2300,19-09-2015,pip fruit,2300_19-09-2015
3,1187,12-12-2015,other vegetables,1187_12-12-2015
4,3037,01-02-2015,whole milk,3037_01-02-2015


In [ ]:
#pivot the table above to convert the items into columns and the transaction into rows

df2 = pd.crosstab(df['single_transaction'], df['itemDescription'])

df2.head()

itemDescription,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
single_transaction,,,,,,,,,,,,,,,,,,,,,
1000_15-03-2015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1000_24-06-2014,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1000_24-07-2015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000_25-11-2015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000_27-05-2015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next step involves encoding all values in the above dataframe to 0 and 1.
if there are multiples of the same item in the same transaction, the value will be encoded to 1.
Note that, market basket analysis does not take purchase frequency into consideration.

In [29]:
def encode(item_frequency):
  res= 0
  if item_frequency > 0:
     res=1
  return res
  basket_input = df2.applymap(encode)

  df.sample(5)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Step 3: Build the Apriori Algorithm for Market Basket Analysis

This stage involves importing Apriori algorithm from the MLXtend Python package and then we use it to discover frequently-bought-together item combinations.

In [32]:
from mlxtend.frequent_patterns import apriori
!pip install apyori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori (basket_input, min_support=0.001, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift")

rules.head()




/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: ignored

In [31]:
rules.sort_values(["support", "confidence","lift"],axis = 0, ascending = False).head(8)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: ignored